# Jensenの不等式
- 上に凸の関数なのか下に凸の関数なのかで不等号の向きが変わるので注意
- ここでは対数尤度関数を考えるので上に凸の関数の場合は記述する
$$
    f(\sum_i \lambda_i x_i) \ge \sum_i \lambda_i f(x_i)
$$

# EMアルゴリズム
- EMアルゴリズムは観測不可能な潜在変数にモデルが依存する場合の最尤推定の計算方法の1つ
- 各変数の定義は以下
$$
    X: 観測変数, Z: 潜在変数, \theta: パラメータ, 対数尤度関数(LL) : \ln p(x|\theta)
$$

LLに潜在変数を導入する。
$$
    LL = \log \sum_z p(X,Z|\theta)
$$
ここに潜在変数についての分布q(Z)を導入し、LLを変分下限とKLに分解できるようにする。変分下限($L(q, \theta)$)はイェンセンの不等式から導出する。

$$
    \log \sum_z p(X,Z|\theta) = \log \sum_Z q(Z) \frac{p(X, Z| \theta)}{q(Z)} \ge \sum_Z q(Z) \log \frac{p(X, Z| \theta)}{q(Z)} = L(q, \theta)
$$

次にLLと変分下限の差を計算するとLLと変分下限の差がカルバックライブラー情報量になってることがわかる
$$
    LL - L(q, \theta) = \log p(X|\theta) - \sum_Z q(Z) \log \frac{p(X, Z| \theta)}{q(Z)} \\
    = \log p(X|\theta) \sum_Z q(Z) - \sum_Z q(Z) \log \frac{p(Z|X, \theta)p(X|\theta)}{q(Z)} \\
    = \sum_Z q(Z) \log p(X|\theta)  - \sum_Z q(Z) \log \frac{p(Z|X, \theta)p(X|\theta)}{q(Z)} \\
    = \sum_Z q(Z) \log \frac{p(Z|X, \theta)}{q(Z)} = KL(q, p)
$$

よって対数尤度は変分下限とKLの和に分解できることがわかる
$$
    LL = L(q, \theta) + KL(q, p)
$$

EMアルゴリズムではEステップで$\theta$を固定し$KL(q,p)=0$となる$q(Z)$を求め、Mステップで$q(Z)$を固定して変分下限$L(q, \theta)$を最大化する$\theta$を求める

Eステップ: $KL(q, p)= 0$ となるのは$q=p$となる時なので
$$
    q(Z) = p(Z|X, \theta^{old})
$$

Mステップ: Eステップで求めた$q(Z)$を使って変分下限を最大化する$\theta$を求める
$$
    L(q, \theta) = \sum_Z q(Z) \log \frac{p(X, Z| \theta)}{q(Z)} = \sum_Z p(Z|X, \theta^{old}) \log \frac{p(X, Z| \theta)}{p(Z|X, \theta^{old})} \\
    = \sum_Z p(Z|X, \theta^{old}) \log p(X, Z| \theta) - p(Z|X, \theta^{old})p(Z|X, \theta^{old}) 
$$
ここで$p(Z|X, \theta^{old})p(Z|X, \theta^{old})$は定数なのでconstとする。
$$
    = \sum_Z p(Z|X, \theta^{old}) \log p(X, Z| \theta) + const = Q(\theta, \theta^{old}) + const
$$

あとはここで定義した$Q(\theta, \theta^{old})$を最大化する$\theta$を求める。(EMアルゴリズムでは$p(X, Z| \theta)$は最適化可能と仮定する)

# 実際にGMMを計算式に従って実装する
- GMMはデータが複数の正規分布のうち1つから生成されたと仮定する
- 複数の正規分布のうちどれだったかを示す潜在変数がZとして定義される
$$
   p(z_k=1)=\pi_k,  p(x|\theta) = \sum_z p(z)p(x|z, \theta) = \sum_z \Pi_k^K(\pi_k N(\mu_k, \Sigma_k))^{z_k} = \sum_k^K \pi_k N(\mu_k, \Sigma_k)
$$

上より対数尤度関数LLは以下になる
$$
    LL = \log p(X|\theta) = \sum_n^N \log (\sum_k^K \pi_k N(\mu_k, \Sigma_k))
$$

Eステップ: $p(Z| X, \theta)$を求めてQ関数を特定する
$$
    p(Z| X, \theta) = \frac{p(X, Z|\theta)}{p(X| \theta)} \\
$$
$$
    p(X, Z|\theta) = \Pi_n^N p(x_n, z_n|\theta) = \Pi_n^N \Pi_k^K (\pi_k N(\mu_k, \Sigma_k))^{z_{nk}}
$$
$$
    p(X |\theta) = \Pi_n^N \sum_k^K \pi_k N(\mu_k, \Sigma_k)
$$

上の式より$p(Z| X, \theta)$が求まる

Q関数は$\sum_Z p(Z|X,\theta^{old}) \log p(X,Z|\theta)$なので$\log p(X,Z|\theta)$を$p(Z|X,\theta^{old})$で重み付けしたものになる。
$$
    \log p(X,Z|\theta) = \sum_n^N \sum_k^K z_{nk}(\log \pi_k + \log N(\mu_k, \Sigma_k))
$$

$\sum_Z p(Z|X,\theta^{old}) $はZに関する期待値になるので$z_{nk}$について計算する
$$
    \sum_Z p(Z|X,\theta^{old}) z_{nk} = \sum_Z \frac{\Pi_k^K (\pi_k N(\mu_k, \Sigma_k))^{z_{nk}}}{\sum_k^K \pi_k N(\mu_k, \Sigma_k)} z_{nk} = \frac{\pi_k N(\mu_k, \Sigma_k))}{\sum_k^K \pi_k N(\mu_k, \Sigma_k)} = \gamma(z_{nk})
$$
よって
$$
    Q = \sum_n^N \sum_k^K \gamma(z_{nk}) (\log \pi_k + \log N(\mu_k, \Sigma_k))
$$

Mステップ: Q関数を$\mu_k, \Sigma_k, \pi_k$で偏微分をして最適なパラメータを求める
$$
    \frac{dQ}{d\mu_k} = \sum_n^N \gamma(z_{nk}) \frac{\log N(\mu_k, \Sigma_k)}{d\mu_k} = 0を解くと
$$
$$
    \mu_k = \frac{1}{N_k} \sum_n^N \gamma(z_{nk})x_n, (N_k = \sum_n^N \gamma(z_{nk}))
$$